In [1]:
import pandas as pd
import numpy as np
from configparser import ConfigParser
from io import StringIO
from azure.storage.blob import BlobServiceClient

Ingest data

In [4]:
parser = ConfigParser()
parser.read('blob_access.ini')

service = BlobServiceClient.from_connection_string(parser.get('key','connection_uri'))
blob = service.get_blob_client('ddrs11', parser.get('key','transaction'))

with open("transaction.csv", "wb") as f:
    data = blob.download_blob()

 df = pd.read_csv(StringIO(data.content_as_text()))


Drop Columns with Similar Meaning and Handle Null values

In [4]:
df_preprocess = df
df_preprocess = df_preprocess.drop(columns = ['collectedamount','newcreditcardflag','visitstatusname','reservationtype','productgroupid','primarywebgenre'],axis=1)
df_preprocess['boxofficerevenueamount'].fillna(df_preprocess['boxofficerevenueamount'].median(), inplace=True)
df_preprocess

,customerentitykey,rentdatekey,rentaldate,transactionid,title_id,kioskclient_id,device,boxofficerevenueamount,primarykioskgenre,movierating,...,creditamount,declinedcreditamount,declinedamount,netrevenue,earnedrevenue,redemptionpoints,redemptionamount,declinedredemptionamount,initialnightcharges,promotioncode
0,362955847,20180407,2018-04-07 19:10:32.493,4219760397,302864,54818,Unknown,17200000.0,Drama,NR,...,0.0,0.0,0.0,1.75,1.75,0.0,0.0,0.0,1.75,7VQU58D9
1,345276985,20180209,2018-02-09 18:55:27.99,4176126052,303362,39064,Unknown,64700000.0,Comedy,R,...,0.0,0.0,0.0,3.00,3.00,0.0,0.0,0.0,1.50,Unknown
2,9505277,20180427,2018-04-27 13:33:00.443,4231565538,304523,68379,Unknown,10700000.0,Drama,R,...,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,1.75,FZ9F757X
3,324189863,20180326,2018-03-26 22:47:48.277,4210325404,300109,72598,Android,75300000.0,Drama,R,...,0.0,0.0,0.0,1.25,1.25,0.0,0.0,0.0,1.50,MARCHMADNESS
4,150519281,20180408,2018-04-08 16:12:28.133,4220535954,303380,10164,iPhone,13500000.0,Comedy,R,...,0.0,0.0,0.0,1.75,1.75,0.0,0.0,0.0,1.75,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7007130,133670902,20180310,2018-03-10 21:13:11.27,4197369830,403215,49994,Unknown,207200000.0,Kids,PG,...,0.0,0.0,0.0,2.00,2.00,0.0,0.0,0.0,2.00,Unknown
7007131,81027912,20180304,2018-03-04 17:12:39.727,4192733364,300147,32216,Unknown,90900000.0,Action & Adventure,R,...,0.0,0.0,0.0,3.00,3.00,0.0,0.0,0.0,1.50,Unknown
7007132,303164071,20180212,2018-02-12 20:38:07.96,4178533839,300128,15970,Unknown,22400000.0,Horror,PG-13,...,0.0,0.0,0.0,3.00,3.00,0.0,0.0,0.0,1.50,Unknown
7007133,430493888,20180410,2018-04-10 19:57:25.46,4221552641,304558,58927,Unknown,40200000.0,Drama,R,...,0.0,0.0,0.0,1.75,1.75,0.0,0.0,0.0,1.75,Unknown


Flag Discount Values

In [5]:
#assign bool values in case of availing the promo code or redemption points
def points (row):
    if row['redemptionamount'] == 0.0 :
        return 0
    else:
        return 1
df_preprocess['red_flag'] = df_preprocess.apply (lambda row: points(row), axis=1)

def promo (row):
    if row['promoamount'] == 0.0 :
        return 0
    else:
        return 1
df_preprocess['promo_flag'] = df_preprocess.apply (lambda row: promo(row), axis=1)

def discount_flag (row):
    if (row['promo_flag'] == 1 or row['red_flag'] == 1):
        return 1
    else:
        return 0
df_preprocess['discount_flag'] = df_preprocess.apply (lambda row: discount_flag(row), axis=1)

filtered_data = df_preprocess



Export Filtered Table

In [ ]:
CONNECT_STR = parser.get('key','connection_uri')
CONTAINER_NAME = parser.get('key','container_name')
input_file = filtered_data
output_blob_name = "Cleaned_Transaction_Data.csv"

blob_service_client = BlobServiceClient.from_connection_string(CONNECT_STR)
blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=output_blob_name)

# Upload file
with open(input_file, "rb") as data:
    blob_client.upload_blob(data=data)

Customer-Transaction Level Data

In [ ]:
customer_transaction_level = filtered_data

#concatination for level of data 
customer_transaction_level['customer-transaction'] = customer_transaction_level['customerentitykey'].astype(str) + ('-') + customer_transaction_level['transactionid'].astype(str)
customer_transaction_level = customer_transaction_level.drop(['transactionid','customerentitykey'],axis = 1) 
response_1.to_csv('customer_transaction_level')

#save as csv to blob
input_file = customer_transaction_level
output_blob_name = "Customer-Transaction_Level_Data.csv"

blob_service_client = BlobServiceClient.from_connection_string(CONNECT_STR)
blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=output_blob_name)

# Upload file
with open(input_file, "rb") as data:
    blob_client.upload_blob(data=data)


File Customer-Transaction_Level_Data.csv Saved Successfully !


Monthly Transaction Table Split

In [8]:
df = filtered_data
df['rentaldate']= pd.to_datetime(df['rentaldate'])

g = df.groupby(pd.Grouper(key='rentaldate', freq='M'))
Monthly_transactions = [group for _,group in g]

Monthly_transactions

[         customerentitykey  rentdatekey              rentaldate  \
 2852585          405856609     20180201 2018-02-01 00:00:12.533   
 5532039          436125149     20180201 2018-02-01 00:00:14.997   
 6070490          436125149     20180201 2018-02-01 00:00:14.997   
 3329487          399781552     20180201 2018-02-01 00:00:36.460   
 4282863          399781552     20180201 2018-02-01 00:00:36.460   
 ...                    ...          ...                     ...   
 3853688            2770332     20180228 2018-02-28 23:59:30.670   
 5412348            2770332     20180228 2018-02-28 23:59:30.670   
 410482           237926572     20180228 2018-02-28 23:59:33.567   
 4664202          269236816     20180228 2018-02-28 23:59:42.380   
 5141742          402206916     20180228 2018-02-28 23:59:55.420   
 
          transactionid  title_id  kioskclient_id    device  \
 2852585     4170447901    303271           26320   Unknown   
 5532039     4170464334    303224           11297   Unkn

Export Monthly Transaction Tables

In [22]:
def monthly_split_export(df):
    '''Splits the data on monthly basis.
    
        Input Parameters: 
        Dataframe : Cleaned data
        Dates in datetime format'''
        
    for i in range(len(df)):
        input_file = df[i]
        output_blob_name = 'Transaction_Data_Month_'+ str(i)+'.csv'
        blob_service_client = BlobServiceClient.from_connection_string(CONNECT_STR)
        blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=output_blob_name)
        with open(input_file, "rb") as data:
            blob_client.upload_blob(data=data)

monthly_split_export(Monthly_transactions)

File Transaction_Data_Month_0.csv Saved Successfully !
File Transaction_Data_Month_1.csv Saved Successfully !
File Transaction_Data_Month_2.csv Saved Successfully !


Transaction on days of campaign

In [23]:

df_camp = df[(df['rentaldate'] >= pd.datetime(2018, 3, 6)) & (df['rentaldate'] <= pd.datetime(2018, 3, 9))]

input_file = df_camp
output_blob_name = 'Campaign_Data.csv'
blob_service_client = BlobServiceClient.from_connection_string(CONNECT_STR)
blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=output_blob_name)
#save as csv
# df_camp.to_csv('transaction_camp.csv')

c:\users\rakshithgkanchan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

